# 内联运行语义函数

[上节](02-running-prompts-from-file.ipynb)展示了如何利用存储在文件上的提示模板来定义语义函数。

这节，我们将展示如何使用语义内核（Semantic Kernel）来定义与您的C#代码内联的函数。这在一些情况下可能会很有用：
- 在运行时利用复杂的规则动态地生成提示语
- 通过编辑C#代码而不是TXT文件来编写提示语。
- 轻松地创建演示，如这个文件

提示模板是使用SK模板语言定义的，它允许引用变量和函数。阅读[这个文档](https://learn.microsoft.com/zh-cn/semantic-kernel/prompt-engineering/configure-prompts)，了解更多关于提示模板的设计决策。

现在我们只使用{{$input}}变量，以后再看更复杂的模板。

几乎所有的语义功能提示都有对{{$input}}的引用，这是用户可以从上下文变量中导入内容的默认方式。

首先准备一个 semantic-kernel 实例，同时加载设置笔记本中定义的[AI后台设置](0-AI-settings.ipynb)：

In [ ]:
#r "nuget: Microsoft.SemanticKernel, 0.14.547.1-preview"

#!import config/Settings.cs

using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.SemanticFunctions;

IKernel kernel = KernelBuilder.Create();

// Configure AI backend used by the kernel
var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();
if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService(model, apiKey, orgId);

创建一个语义函数，使用摘要内容：

In [ ]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";

然后配置 prompt，例如允许有一些创造力和足够数量的 tokens。

In [ ]:
var promptConfig = new PromptTemplateConfig
{
    Completion =
    {
        MaxTokens = 2000,
        Temperature = 0.2,
        TopP = 0.5,
    }
};

下面的代码准备了一个模板的实例，传入了上面的TXT和配置，以及其他一些参数（如何渲染TXT以及模板如何访问其他函数）。

In [ ]:
var promptTemplate = new PromptTemplate(
    skPrompt,                        // Prompt template defined in natural language
    promptConfig,                    // Prompt configuration
    kernel                           // SK instance
);

让我们把提示模板转换成内核可以执行的函数：

In [ ]:
var functionConfig = new SemanticFunctionConfig(promptConfig, promptTemplate);

var summaryFunction = kernel.RegisterSemanticFunction("MySkill", "Summary", functionConfig);

设置一些内容进行总结，以下是关于古希腊诗人 Demo 的摘录，摘自维基百科。

In [ ]:
var input = """
Demo (ancient Greek poet)
From Wikipedia, the free encyclopedia
Demo or Damo (Greek: Δεμώ, Δαμώ; fl. c. AD 200) was a Greek woman of the Roman period, known for a single epigram, engraved upon the Colossus of Memnon, which bears her name. She speaks of herself therein as a lyric poetess dedicated to the Muses, but nothing is known of her life.[1]
Identity
Demo was evidently Greek, as her name, a traditional epithet of Demeter, signifies. The name was relatively common in the Hellenistic world, in Egypt and elsewhere, and she cannot be further identified. The date of her visit to the Colossus of Memnon cannot be established with certainty, but internal evidence on the left leg suggests her poem was inscribed there at some point in or after AD 196.[2]
Epigram
There are a number of graffiti inscriptions on the Colossus of Memnon. Following three epigrams by Julia Balbilla, a fourth epigram, in elegiac couplets, entitled and presumably authored by "Demo" or "Damo" (the Greek inscription is difficult to read), is a dedication to the Muses.[2] The poem is traditionally published with the works of Balbilla, though the internal evidence suggests a different author.[1]
In the poem, Demo explains that Memnon has shown her special respect. In return, Demo offers the gift for poetry, as a gift to the hero. At the end of this epigram, she addresses Memnon, highlighting his divine status by recalling his strength and holiness.[2]
Demo, like Julia Balbilla, writes in the artificial and poetic Aeolic dialect. The language indicates she was knowledgeable in Homeric poetry—'bearing a pleasant gift', for example, alludes to the use of that phrase throughout the Iliad and Odyssey.[a][2] 
""";

执行摘要函数：

In [ ]:
var summary = await summaryFunction.InvokeAsync(input);

Console.WriteLine(summary);

上面的代码显示了所有的步骤，以了解该函数是如何一步步组成的。然而，内核还包括一些辅助工具，以更简洁地实现同样的功能。

上述同样的函数可以用更少的代码创建：

In [ ]:
string skPrompt = """
{{$input}}

Summarize the content above.
""";
var summaryFunction = kernel.CreateSemanticFunction(skPrompt, maxTokens: 2000, temperature: 0.2, topP: 0.5);

var summary = await summaryFunction.InvokeAsync(input);
Console.WriteLine(summary);

下面再举一个例子，说明如何编写一个内联语义功能，为一段文字提供一个TLDR。

In [ ]:
var kernel = Microsoft.SemanticKernel.Kernel.Builder.Build();

var (useAzureOpenAI, model, azureEndpoint, apiKey, orgId) = Settings.LoadFromFile();

if (useAzureOpenAI)
    kernel.Config.AddAzureTextCompletionService(model, azureEndpoint, apiKey);
else
    kernel.Config.AddOpenAITextCompletionService(model, apiKey, orgId);


string skPrompt = @"
{{$input}}

Give me the TLDR in 5 words.
";

var textToSummarize = @"
    1) A robot may not injure a human being or, through inaction,
    allow a human being to come to harm.

    2) A robot must obey orders given it by human beings except where
    such orders would conflict with the First Law.

    3) A robot must protect its own existence as long as such protection
    does not conflict with the First or Second Law.
";

var tldrFunction = kernel.CreateSemanticFunction(skPrompt, maxTokens: 200, temperature: 0, topP: 0.5);

var summary = await tldrFunction.InvokeAsync(textToSummarize);

Console.WriteLine(summary);

// Output => Robots must not harm humans.